In [32]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import pandas as pd
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import os

# Initialize Pinecone


# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
import os
import faiss
import numpy as np
import pickle

# Load the FAISS index
storage_directory = r'C:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\stored_data'
index = faiss.read_index(os.path.join(storage_directory, 'faiss_index.bin'))

# Load document texts
with open(os.path.join(storage_directory, 'documents.pkl'), 'rb') as f:
    documents = pickle.load(f)

# Load metadata
with open(os.path.join(storage_directory, 'metadata.pkl'), 'rb') as f:
    file_metadata = pickle.load(f)

# Ensure documents and metadata are loaded correctly
print(f"Loaded {len(documents)} documents.")
print(f"Loaded {len(file_metadata)} metadata entries.")

# Function to retrieve text and documents based on a query
def retrieve_text_and_doc(query, k=2):
    # Step 1: Compute the query embedding
    query_embedding = embedding_model.encode([query], convert_to_tensor=False)
    
    # Check embedding size and FAISS index dimensionality
    assert query_embedding.shape[1] == index.d, f"Embedding size {query_embedding.shape[1]} does not match FAISS index dimensionality {index.d}"

    # Step 2: Perform the search in the FAISS index
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)

    # Step 3: Retrieve the corresponding documents and metadata
    retrieved_docs = []
    retrieved_metadata = []
    
    for i in indices[0]:
        if i < len(documents):
            retrieved_docs.append(documents[i])
            retrieved_metadata.append(file_metadata[i])
        else:
            print(f"Warning: Index {i} is out of range for documents. Total documents: {len(documents)}.")

    return retrieved_docs, retrieved_metadata, distances[0]

# Example usage
query = input("Enter your query: ")
retrieved_docs, retrieved_metadata, distances = retrieve_text_and_doc(query)

# Join and display retrieved text and metadata
retrieved_text = " ".join(retrieved_docs)

for doc, meta, dist in zip(retrieved_docs, retrieved_metadata, distances):
    print(f"Metadata: {meta}, Distance: {dist}, Document: {doc[:100]}...")


Loaded 9 documents.
Loaded 9 metadata entries.
Metadata: Doc 1: Synopsis Architect-Engineer (A-E) Services Indefinite Delivery/Indefinite Quantity (IDIQ) Contract For, Distance: 1.59644615650177, Document: Synopsis Architect-Engineer (A-E) Services Indefinite Delivery/Indefinite Quantity (IDIQ) Contract F...
Metadata: Doc 8:Name: Abhinav Kasubojula Internship Company: Kenall inc Role: AI Intern, Distance: 1.6353421211242676, Document: Name: Abhinav Kasubojula Internship Company: Kenall inc Role: AI Intern Internship Supervisor: Ganes...


In [34]:
retrived_text = " ".join(retrieved_docs)
print(retrived_text)

Synopsis Architect-Engineer (A-E) Services Indefinite Delivery/Indefinite Quantity (IDIQ) Contract For Army Facilities Components System (AFCS) 100% Small Business Set Aside W912DQ24R4019
1. CONTRACT INFORMATION: This contract is being procured in accordance with 40 U.S.C. Chap 11, Selection of Architects and Engineers (formerly known as the Brooks Architect Engineer) Act as implemented in FAR Subpart 36.6 and DFARS Subpart 236.6. One Firm will be selected for negotiation based on demonstrated competence and qualifications for the required work. This A-E IDC is required to support projects assigned to the Kansas City District (NWK), U.S. Army Corps of Engineers related to the Army Facilities Components System (AFCS). This synopsis has a target of one (1) contract restricted to small business. The total value of all task orders issued to the small business contract will not exceed $20,000,000.00. Under a separate synopsis, a target of one (1) contract will be open to all businesses rega

In [35]:
import requests
import json

# Initialize an empty list to store previous questions and answers
memory = []

# Pinecone text retrieval part (you've done this)
def generate_response(cx, q):
    global memory

    # Retrieved text as context from Pinecone
    retrieved_text = cx

    # Prepare the prompt with memory of past interactions
    memory_prompt = "\n".join(memory)  # Join the previous interactions
    question = q
    prompt = f"Here is some context:\n{retrieved_text}\n{memory_prompt}\n\nNow, based on this, can you answer the following question: {question}"

    # Ollama API call to generate a response
    url = "http://localhost:11434/api/generate"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3.1:latest",
        "prompt": prompt,
        "stream": False  # Disable streaming
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    # Check the response from the model
    if response.status_code == 200:
        result = response.json()
        ans = result['response']
        print("Model response:", ans)

        # Store the current question and answer in memory
        memory.append(f"Q: {question}\nA: {ans}")
        if len(memory) > 5:  
            memory = memory[-5:]

        return ans
    else:
        print("Error:", response.text)
        return None

# Example usage:
context = retrived_text
question = query
answer = generate_response(context, question)


Model response: Based on the text, it appears that there is no mention of establishing a "communication pathway" as the first step. However, I can try to help you with the actual first step mentioned in the instructions.

According to the text, the first step is:

1. Navigate to https://piee.eb.mil/


In [4]:
answer = generate_response("did you remember your last 3 responses?", "?")

In [5]:
print(query)

notice id of solicitation W912EP22R0042. give me contract details
